In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import rand as sprand
import torch
import torch.nn as nn
import torch.nn.functional as F

# Make up some random explicit feedback ratings
# and convert to a numpy array
n_users = 1000
n_items = 1000
ratings = sprand(n_users, n_items, 
                 density=0.01, format='csr')
ratings.data = (np.random.randint(1, 5, 
                                  size=ratings.nnz)
                          .astype(np.float64))
ratings = ratings.toarray()

In [9]:
df = pd.read_csv('okved_sum_with_private.csv')

In [16]:
df = df[[col for col in df.columns if col not in ['hash_inn', 'okved2', 'region', 'total_dt', 'cnt_not_null_dt'] and 'share' not in col]]

In [17]:
df = df.iloc[: 1000,]

In [18]:
df.fillna(0, inplace=True)

In [19]:
df.shape

(1000, 162)

In [20]:
df.head()

,0_sum_kt,1_sum_kt,10_sum_kt,11_sum_kt,12_sum_kt,13_sum_kt,14_sum_kt,15_sum_kt,16_sum_kt,17_sum_kt,...,72_sum_dt_dt,73_sum_dt_dt,74_sum_dt_dt,75_sum_dt_dt,76_sum_dt_dt,77_sum_dt_dt,78_sum_dt_dt,79_sum_dt_dt,8_sum_dt_dt,9_sum_dt_dt
0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.196,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=True)
        
    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [23]:
n_users, n_items = df.shape[0], df.shape[1]

In [24]:
model = MatrixFactorization(n_users, n_items, n_factors=20)

In [39]:
model

MatrixFactorization(
  (user_factors): Embedding(1000, 20, sparse=True)
  (item_factors): Embedding(162, 20, sparse=True)
)

In [25]:
loss_func = torch.nn.MSELoss()

In [26]:
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=1e-6) # learning rate

In [31]:
ratings = df.values

In [32]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

for row, col in zip(*(rows, cols)):
    # Turn data into tensors
    rating = torch.FloatTensor([ratings[row, col]])
    row = torch.LongTensor([row])
    col = torch.LongTensor([col])
    
    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)
    
    # Backpropagate
    loss.backward()
    
    # Update the parameters
    optimizer.step()

In [33]:
# basic matrix factorization model without bias term
class Recommender(nn.Module):
    def __init__(self, num_users, num_artists, num_factors):
        super().__init__()
        self.u = nn.Embedding(num_users, num_factors)
        self.a = nn.Embedding(num_artists, num_factors)
        self.u.weight.data.uniform_(-.01, .01)
        self.a.weight.data.uniform_(-.01, .01)

    def forward(self, cats, conts):
        users, artists = cats[:,0], cats[:,1]
        us, art = self.u(users), self.a(artists)
        return (us*art).sum(1)

In [34]:
class BiasedMatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=True)
        self.user_biases = torch.nn.Embedding(n_users, 
                                              1,
                                              sparse=True)
        self.item_biases = torch.nn.Embedding(n_items,
                                              1,
                                              sparse=True)
        
    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (self.user_factors(user) * self.item_factors(item)).sum(dim=1, keepdim=True)
        return pred.squeeze()

In [35]:
reg_loss_func = torch.optim.SGD(model.parameters(), lr=1e-6,
                                weight_decay=1e-5)

In [36]:
reg_loss_func

SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-06
    momentum: 0
    nesterov: False
    weight_decay: 1e-05
)

In [37]:
adagrad_loss = torch.optim.Adagrad(model.parameters(), lr=1e-6)

In [38]:
adagrad_loss

Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 1e-06
    lr_decay: 0
    weight_decay: 0
)